<a href="https://colab.research.google.com/github/gsdos1984-sudo/simulador-moldeo/blob/main/EPP_Caption_by_MAVG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
# ================== DEPENDENCIAS + AUTH ==================
!pip install -q gradio plotly pandas gspread gspread_dataframe openpyxl

from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default
creds, _ = default(scopes=['https://www.googleapis.com/auth/spreadsheets',
                           'https://www.googleapis.com/auth/drive'])
gc = gspread.authorize(creds)

# ================== HOJA DE CÁLCULO (Google Sheets) ==================
SHEET_NAME = "EPP_APP_DATA"   # <- cambia si deseas otro nombre
ADMIN_KEY = "EPPADMIN"        # <<< CAMBIA ESTA CLAVE PARA EXPORTAR EXCEL

BASE_HEADER = [
    "ts","shift","machine","operator","product",
    "steam_bar","xsteam_fix_bar","xsteam_mobile_bar","autoclave_bar",
    "air_pre_bar","water_pre_bar","cooling_tower","cycle_sec",
    "Filling1-1 (sec) ","Filling1-1(bar) Setting(0.1-6 bar)","Filling2-1 (sec)","Filling2-1 (0.1-6bar)","Pre-Blowing (sec)","Blowback (sec)",
    "cooling_fix_sec(10-100sec)","cooling_mobile_sec(10-100sec)","purge_air_sec(1-10sec)","drain_sec(1-10sec)","foma_sec(1-10sec)",
    "pre_blow_sec","ejector_air_sec","crack_air_sec","holding_press_bar","transfer_press_bar",
    "wet_w_min","wet_w_max","qty_dim_chk",
    "e_stop_ok","ppe_ok","light_curtain_ok","door_ok","loto_ok","five_s_ok",
    "downtime_min","downtime_code","scrap_qty","scrap_code","comments"
]

try:
    sh = gc.open(SHEET_NAME)
except gspread.SpreadsheetNotFound:
    sh = gc.create(SHEET_NAME)
ws = sh.sheet1

def ensure_header():
    vals = ws.get_all_values()
    if not vals:
        ws.append_row(BASE_HEADER)
    else:
        if vals[0] != BASE_HEADER:
            ws.clear()
            ws.append_row(BASE_HEADER)
ensure_header()

# ================== APP ==================
import gradio as gr
import pandas as pd
import plotly.express as px
from datetime import datetime

DOWNTIME_CODES = [
    "Prep (Scheduled) / Preparación",
    "Rest (Scheduled) / Descanso",
    "Utility - Low Steam Pressure / Baja presión de vapor",
    "Utility - Low Air Pressure / Baja presión de aire",
    "Machine - Other / Máquina - Otros",
    "Mold - Dirty Mold / Molde sucio",
    "Mold - Ejector Rod Alignment / Alineación expulsor",
    "Bead Transfer / Material Line Clog / Atasco de línea",
    "5S / Limpieza"
]
SCRAP_CODES = [
    "Adhesion / Adhesión","Dimension / Dimensión","Underfilling / Falta de llenado",
    "Demoulding / Desmoldeo","Warping / Pandeo","Mold - Set up / Arranque",
    "Process Setting / Valores de Proceso"
]

def sheet_to_df(): return pd.DataFrame(ws.get_all_records())
def df_append_row(row): ws.append_row([row.get(h,"") for h in BASE_HEADER], value_input_option="USER_ENTERED")
def empty_dash(): return gr.update(visible=False), "", None, None, None, None, pd.DataFrame()

# ---------- Dashboard ----------
def load_dashboard():
    df = sheet_to_df()
    if df.empty: return empty_dash()
    for col in ["downtime_min","scrap_qty","cycle_sec"]:
        if col in df.columns: df[col] = pd.to_numeric(df[col], errors="coerce")

    total_rows, total_downtime, total_scrap = len(df), int(df["downtime_min"].sum()), int(df["scrap_qty"].sum())
    avg_cycle = round(df["cycle_sec"].mean(),1) if "cycle_sec" in df and not df["cycle_sec"].dropna().empty else 0
    kpi = f"**Registros/Records:** {total_rows} | **Downtime (min):** {total_downtime} | **Scrap (pcs):** {total_scrap} | **Cycle avg (s):** {avg_cycle}"

    fig_code, fig_scrap, fig_shift, fig_pareto = None, None, None, None
    if "downtime_code" in df:
        by_code=df.groupby("downtime_code")["downtime_min"].sum().reset_index()
        fig_code=px.bar(by_code.sort_values("downtime_min",ascending=False),x="downtime_code",y="downtime_min",title="Downtime by Code",text="downtime_min")
    if "scrap_code" in df:
        by_scrap=df.groupby("scrap_code")["scrap_qty"].sum().reset_index()
        fig_scrap=px.bar(by_scrap.sort_values("scrap_qty",ascending=False),x="scrap_code",y="scrap_qty",title="Scrap by Reason",text="scrap_qty")
    if "shift" in df:
        by_shift=df.groupby("shift").size().reset_index(name="reg")
        fig_shift=px.bar(by_shift,x="shift",y="reg",title="Records by Shift",text="reg")
    if "machine" in df:
        by_machine=df.groupby("machine")["downtime_min"].sum().reset_index().sort_values("downtime_min",ascending=False)
        if by_machine["downtime_min"].sum()>0:
            by_machine["cum%"]=100*by_machine["downtime_min"].cumsum()/by_machine["downtime_min"].sum()
        fig_pareto=px.bar(by_machine,x="machine",y="downtime_min",title="Pareto Downtime by Machine",text="downtime_min")
        fig_pareto.add_scatter(x=by_machine["machine"],y=by_machine["cum%"],mode="lines+markers",name="Cum %",yaxis="y2")
        fig_pareto.update_layout(yaxis=dict(title="min"),yaxis2=dict(title="Cum %",overlaying="y",side="right",range=[0,100]))
    return gr.update(visible=True),kpi,fig_code,fig_scrap,fig_shift,fig_pareto,df.tail(20)

# ---------- Submit ----------
def clear_values(): return [gr.update(value="") for _ in BASE_HEADER[1:]]  # limpia todos menos ts
def submit_form(*args):
    data={h:(str(v) if v not in [None,""] else "") for h,v in zip(BASE_HEADER[1:],args)}
    data["ts"]=datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    df_append_row(data)
    visible,kpi,fc,fs,fh,fp,tail=load_dashboard()
    return "✅ Registro enviado",visible,kpi,fc,fs,fh,fp,tail,*clear_values()

# ---------- Export Excel ----------
def export_excel(admin_key):
    if admin_key!=ADMIN_KEY: return "❌ Clave incorrecta",None
    df=sheet_to_df()
    if df.empty: return "⚠️ No hay datos",None
    by_code=df.groupby("downtime_code")["downtime_min"].sum().reset_index()
    by_scrap=df.groupby("scrap_code")["scrap_qty"].sum().reset_index()
    by_machine=df.groupby("machine")["downtime_min"].sum().reset_index()
    ts=datetime.now().strftime("%Y%m%d_%H%M%S")
    path=f"/content/EPP_export_{ts}.xlsx"
    with pd.ExcelWriter(path,engine="openpyxl") as w:
        df.to_excel(w,index=False,sheet_name="Data")
        by_code.to_excel(w,index=False,sheet_name="Downtime_by_Code")
        by_scrap.to_excel(w,index=False,sheet_name="Scrap_by_Reason")
        by_machine.to_excel(w,index=False,sheet_name="Pareto_by_Machine")
    return "✅ Excel generado",path

# ================== UI ==================
with gr.Blocks(theme=gr.themes.Soft(), title="EPP – Captura & Dashboard") as demo:
    gr.Markdown("## 📲 EPP – Captura móvil + Dashboard")

    with gr.Tab("Captura / Capture"):
        inputs=[]
        for h in BASE_HEADER[1:]:
            if "ok" in h: comp=gr.Checkbox(label=h,value=True)
            elif "downtime_min" in h or "scrap_qty" in h: comp=gr.Number(label=h,value=0)
            elif "code" in h: comp=gr.Dropdown(choices=DOWNTIME_CODES if "downtime" in h else SCRAP_CODES,label=h)
            else: comp=gr.Textbox(label=h)
            inputs.append(comp)
        status=gr.Textbox(label="Estado / Status",interactive=False)
        submit=gr.Button("Enviar / Submit")

    with gr.Tab("Dashboard"):
        dash_group=gr.Group(visible=False)
        with dash_group:
            kpi=gr.Markdown(); fc=gr.Plot(); fs=gr.Plot(); fh=gr.Plot(); fp=gr.Plot()
            tail=gr.Dataframe(row_count=(5,"dynamic"),wrap=True,label="Últimos registros")
        refresh=gr.Button("🔄 Actualizar / Refresh")
        refresh.click(load_dashboard,outputs=[dash_group,kpi,fc,fs,fh,fp,tail])
        auto=gr.Timer(5.0); auto.tick(load_dashboard,outputs=[dash_group,kpi,fc,fs,fh,fp,tail])

    with gr.Tab("Admin"):
        key_in=gr.Textbox(label="Clave Admin",type="password")
        btn=gr.Button("Exportar Excel")
        est=gr.Textbox(label="Estado",interactive=False)
        f=gr.File(label="Archivo Excel",interactive=False)
        btn.click(export_excel,inputs=[key_in],outputs=[est,f])

    submit.click(submit_form,inputs=inputs,outputs=[status,dash_group,kpi,fc,fs,fh,fp,tail,*inputs])
    demo.load(load_dashboard,outputs=[dash_group,kpi,fc,fs,fh,fp,tail])

demo.launch(share=True)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://cbfd7a498722eac59a.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
